Mount Google Drive env. for importing the dataset

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


# Append the directory to our python path using sys

Now we are able to import stuff from that directory.

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/ANFIS-imgSatellitari')

In [3]:
%cd /content/drive/MyDrive/ANFIS-imgSatellitari/

/content/drive/MyDrive/ANFIS-imgSatellitari


Import all the libraries and modules.



In [4]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import torch
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from training import train_model
from testMod import test_model
import anfis
from membership import *
from utility import load_model, plot_import, split_dataset

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


cpu
Begin training.


# Test 1

In the following test we choose to configure the ANFIS training with:


*   Nr. 2 Fuzzy Sets
*   32 of Batch size
*   Nr. 20 of epochs



In [ ]:
dataset = 'datasets/reduced4ClassV3/reducedV3.csv'
model = None
n_terms = 2 #IMPOSTA NUMERO DI FUZZY SET
batch_size = 32
num_categories = 4 #IMPOSTA NUMERO DI CLASSI
epoch = 20
model_l = False #SEMPRE A FALSE
hybrid = False #SEMPRE A FALSE
i = 0
lista_acc = []
k_fold = False

# Make the classes available via (controlled) reflection:
get_class_for = {n: globals()[n]
                 for n in ['BellMembFunc',
                           'GaussMembFunc',
                           'TriangularMembFunc',
                           'TrapezoidalMembFunc',
                           ]}

# DEFINIRE LA MEMBERSHIP FUNCTION ATTUALMENTE E' ABILITATA SOLO GAUSSIANA E TRIANGOLARE
membership_function = get_class_for['GaussMembFunc']

d_data, d_target = split_dataset(dataset)


# Split train into trainval-test
X_trainval, X_test, y_trainval, y_test = train_test_split(d_data, d_target, test_size=0.3, shuffle=True,
                                                          stratify=d_target, random_state=42)

# Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, shuffle=True,
                                               stratify=y_trainval, random_state=42)


model = train_model(model, X_train, y_train, X_val, y_val, n_terms, num_categories,
                                   batch_size, epoch, model_l, hybrid, membership_function, i)

torch.save(model, 'models/G_model_geo_' + str(i) + '.h5')

model = torch.load('models/G_model_geo_0.h5')

test_model(model, X_test, y_test, num_categories, k_fold, i, lista_acc)

model = torch.load('models/G_model_geo_0.h5')

X_test = torch.Tensor(X_test)

y_pred = model(X_test)

cat_act = torch.argmax(y_pred, dim=1)

#PREDIZIONE SUI DATI DI TEST
print(cat_act)

cm = confusion_matrix(y_test, cat_act)
print(cm)

cl = classification_report(y_test, cat_act)
print(cl)





Model - Pre-training
AnfisNet(
  Rule  0: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  1: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  2: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  3: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  4: IF x0 is mf0 and x1 is mf1 and x2 is mf0 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  5: IF x0 is mf0 and x1 is mf1 and x2 is mf0 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  6: IF x0 is mf0 and x1 is mf1 and x2 is mf1 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  7: IF x0 is mf0 and x1 is mf1 and x2 is mf1 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  8: IF x0 is mf1 and x1 is mf0 and x2 is mf0 and x3 is mf0
           

best epoch 1
Epoch 001: | Train Loss: 0.79234869 | Val Loss: 0.74655509 | Train Acc: 0.65677446| Val Acc: 0.66744496
best epoch 2
Epoch 002: | Train Loss: 0.70189033 | Val Loss: 0.68850253 | Train Acc: 0.70611867| Val Acc: 0.70287283
best epoch 3
Epoch 003: | Train Loss: 0.65701054 | Val Loss: 0.65606453 | Train Acc: 0.73128433| Val Acc: 0.72535148
best epoch 4
Epoch 004: | Train Loss: 0.63368048 | Val Loss: 0.63935318 | Train Acc: 0.74234207| Val Acc: 0.73588333
best epoch 5
Epoch 005: | Train Loss: 0.62151199 | Val Loss: 0.62992393 | Train Acc: 0.74712399| Val Acc: 0.74093912
best epoch 6
Epoch 006: | Train Loss: 0.61425032 | Val Loss: 0.62379477 | Train Acc: 0.74982965| Val Acc: 0.74409432
best epoch 7
Epoch 007: | Train Loss: 0.60930839 | Val Loss: 0.61941415 | Train Acc: 0.75135550| Val Acc: 0.74605561
best epoch 8
Epoch 008: | Train Loss: 0.60568548 | Val Loss: 0.61613722 | Train Acc: 0.75239635| Val Acc: 0.74752659
best epoch 9
Epoch 009: | Train Loss: 0.60293170 | Val Loss: 0.6

<Figure size 432x288 with 0 Axes>

# Test 2

In the following test we choose to configure the ANFIS training with:


*   Nr. 3 Fuzzy Sets
*   32 of Batch size
*   Nr. 20 of epochs



In [5]:
dataset = 'datasets/reduced4ClassV3/reducedV3.csv'
model = None
n_terms = 3 #IMPOSTA NUMERO DI FUZZY SET
batch_size = 32
num_categories = 4 #IMPOSTA NUMERO DI CLASSI
epoch = 20
model_l = False #SEMPRE A FALSE
hybrid = False #SEMPRE A FALSE
i = 0
lista_acc = []
k_fold = False

# Make the classes available via (controlled) reflection:
get_class_for = {n: globals()[n]
                 for n in ['BellMembFunc',
                           'GaussMembFunc',
                           'TriangularMembFunc',
                           'TrapezoidalMembFunc',
                           ]}

# DEFINIRE LA MEMBERSHIP FUNCTION ATTUALMENTE E' ABILITATA SOLO GAUSSIANA E TRIANGOLARE
membership_function = get_class_for['GaussMembFunc']

d_data, d_target = split_dataset(dataset)


# Split train into trainval-test
X_trainval, X_test, y_trainval, y_test = train_test_split(d_data, d_target, test_size=0.3, shuffle=True,
                                                          stratify=d_target, random_state=42)

# Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, shuffle=True,
                                               stratify=y_trainval, random_state=42)


model = train_model(model, X_train, y_train, X_val, y_val, n_terms, num_categories,
                                   batch_size, epoch, model_l, hybrid, membership_function, i)

torch.save(model, 'models/G_model_geo_' + str(i) + '.h5')

model = torch.load('models/G_model_geo_0.h5')

test_model(model, X_test, y_test, num_categories, k_fold, i, lista_acc)

model = torch.load('models/G_model_geo_0.h5')

X_test = torch.Tensor(X_test)

y_pred = model(X_test)

cat_act = torch.argmax(y_pred, dim=1)

#PREDIZIONE SUI DATI DI TEST
print(cat_act)

cm = confusion_matrix(y_test, cat_act)
print(cm)

cl = classification_report(y_test, cat_act)
print(cl)





Model - Pre-training
AnfisNet(
  Rule  0: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  1: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  2: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf2
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  3: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  4: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  5: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf2
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  6: IF x0 is mf0 and x1 is mf0 and x2 is mf2 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  7: IF x0 is mf0 and x1 is mf0 and x2 is mf2 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  8: IF x0 is mf0 and x1 is mf0 and x2 is mf2 and x3 is mf2
           

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

RISULTATI TEST
150254 of 196608 correct (accuracy=76.42%)
tensor([3, 0, 3,  ..., 2, 2, 1])
[[28373  7664  6234    12]
 [ 6368 16105  9049   123]
 [ 4309  5992 86818  1289]
 [    2     0  5312 18958]]
              precision    recall  f1-score   support

         0.0       0.73      0.67      0.70     42283
         1.0       0.54      0.51      0.52     31645
         2.0       0.81      0.88      0.84     98408
         3.0       0.93      0.78      0.85     24272

    accuracy                           0.76    196608
   macro avg       0.75      0.71      0.73    196608
weighted avg       0.76      0.76      0.76    196608



# Test 3

In the following test we choose to configure the ANFIS training with:


*   Nr. 4 Fuzzy Sets
*   32 of Batch size
*   Nr. 20 of epochs



In [ ]:
dataset = 'datasets/reduced4ClassV3/reducedV3.csv'
model = None
n_terms = 4 #IMPOSTA NUMERO DI FUZZY SET
batch_size = 32
num_categories = 4 #IMPOSTA NUMERO DI CLASSI
epoch = 20
model_l = False #SEMPRE A FALSE
hybrid = False #SEMPRE A FALSE
i = 0
lista_acc = []
k_fold = False

# Make the classes available via (controlled) reflection:
get_class_for = {n: globals()[n]
                 for n in ['BellMembFunc',
                           'GaussMembFunc',
                           'TriangularMembFunc',
                           'TrapezoidalMembFunc',
                           ]}

# DEFINIRE LA MEMBERSHIP FUNCTION ATTUALMENTE E' ABILITATA SOLO GAUSSIANA E TRIANGOLARE
membership_function = get_class_for['GaussMembFunc']

d_data, d_target = split_dataset(dataset)


# Split train into trainval-test
X_trainval, X_test, y_trainval, y_test = train_test_split(d_data, d_target, test_size=0.3, shuffle=True,
                                                          stratify=d_target, random_state=42)

# Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, shuffle=True,
                                               stratify=y_trainval, random_state=42)


model = train_model(model, X_train, y_train, X_val, y_val, n_terms, num_categories,
                                   batch_size, epoch, model_l, hybrid, membership_function, i)

torch.save(model, 'models/G_model_geo_' + str(i) + '.h5')

model = torch.load('models/G_model_geo_0.h5')

test_model(model, X_test, y_test, num_categories, k_fold, i, lista_acc)

model = torch.load('models/G_model_geo_0.h5')

X_test = torch.Tensor(X_test)

y_pred = model(X_test)

cat_act = torch.argmax(y_pred, dim=1)

#PREDIZIONE SUI DATI DI TEST
print(cat_act)

cm = confusion_matrix(y_test, cat_act)
print(cm)

cl = classification_report(y_test, cat_act)
print(cl)





Model - Pre-training
AnfisNet(
  Rule  0: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  1: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  2: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf2
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  3: IF x0 is mf0 and x1 is mf0 and x2 is mf0 and x3 is mf3
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  4: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf0
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  5: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf1
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  6: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf2
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  7: IF x0 is mf0 and x1 is mf0 and x2 is mf1 and x3 is mf3
           THEN [[0.0], [0.0], [0.0], [0.0]]
  Rule  8: IF x0 is mf0 and x1 is mf0 and x2 is mf2 and x3 is mf0
           

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

RISULTATI TEST
151733 of 196608 correct (accuracy=77.18%)
tensor([3, 0, 3,  ..., 2, 2, 1])
[[28222  8032  6006    23]
 [ 5709 17178  8648   110]
 [ 4190  6187 86998  1033]
 [    5     0  4932 19335]]
              precision    recall  f1-score   support

         0.0       0.74      0.67      0.70     42283
         1.0       0.55      0.54      0.54     31645
         2.0       0.82      0.88      0.85     98408
         3.0       0.94      0.80      0.86     24272

    accuracy                           0.77    196608
   macro avg       0.76      0.72      0.74    196608
weighted avg       0.77      0.77      0.77    196608

